# GregoriOCR

### 0. Download gregorio data.
### 1. Convert image to numpy array.
### 2. Use thin strip to scan across image looking for notes.
    a. The strip is the height of the image and 1 pixel wide.
    b. If the strip sees a "tower" of dark pixels (taller than some threshold height, e.g., 10 pixels tall), that's a potential note.
    c. If the subsequent strips also see "towers" of dark pixels at that same height, we classify decide a note is there.  (E.g., we had 10 strips in a row all see a tower of dark pixels at the same height, so we decide that there must be a note there.)
### 3. Calculate the center pixel of the note we found.
### 4. Classify double and triple notes.
    a. When we find two notes that are very close together, we classify them as a double note.
    b. E.g., our threshold is 10, and two centers are at (20, 56) and (25, 83).  25 - 20 = 5 < 10, so the notes are very close together and must be double notes.
    c. To classify triple notes, we just need to look one note further.  If we find another note that is very close to the double note, then we classify that as a triple note.
### 5. Find pitch of first note in staff.
    a. Use CNN to find pitch of that note?
    b. If we use a CNN, we would cut out a window around the first note.  (The window would be the height of the image and the width of the note.  Use CNN to classify that note.
    c. Note: To make this easier for the CNN, if the first note is a double note, we can instead use the first single note in the staff.
### 6. Classify pitches of all other notes based on relative height to first note.
    a. We decide some threshold distance for when two notes are at different pitches, e.g., 10 pixel height difference.
    b. E.g., first note is at (20, 50) and second note is at (20, 80).  80 - 50 = 30, so second note is 3 pitches above first note.
### 7. Classify note types.
    a. Cut out square around each center, and use CNN to classify note type.
### 8. Combine pitch and note type information to make list of all notes in staff.
### 9. Convert list to gabc output.
### 10. Evaluate accuracy.

In [1]:
#Libraries
from keras.layers import Conv2D, Dense, Flatten
from keras.models import Sequential
import numpy as np
from PIL import Image

# 1. Convert image to numpy array.

In [ ]:
img = Image.open("image.png")
arr = np.array(img)

#Old:
#data = np.load('./data/cats/cats.npz') #This is set for a different dataset

In [ ]:
#For looking at the file contents
lst = data.files
for item in lst:
    print(item)
    print(data[item])

In [ ]:
X_train, y_train = data['Xtrain'].transpose(), data['Ytrain'].transpose()
X_test, y_test = data['Xtest'].transpose(), data['Ytest'].transpose()

#Reshape the data from a 1D vector to a 2D matrix.
#This represents a grid of pixel coordinates.
#X_train.shape[0] returns our number of observations in X_train.
#Parameters of .reshape():
    #.reshape([number of observations in dataset, height in pixels, width in pixels, number of color dimensions])
X_train = X_train.reshape([X_train.shape[0], 64, 64, 1])

In [ ]:
#Display
display(X_train.shape)
display(y_train.shape)
display(X_test.shape)
display(y_test.shape)

In [ ]:
#ONE-HOT ENCODE y_train

#Use one-hot encoding to transform y_train into a matrix where each column represents a different possible outcome.
#This allows the model to predict more than 2 classes.
#In our case, y_train will be m x n, where m is the number of observations and n is the number of possible outcomes (note types).

one_hot_encoding = pd.get_dummies(y_train)
y_train = one_hot_encoding

display(y_train)

# 2. Use thin strip to scan across image looking for notes.
a. The strip is the height of the image and 1 pixel wide.

b. If the strip sees a "tower" of dark pixels (taller than some threshold height, e.g., 10 pixels tall), that's a potential note.

c. If the subsequent strips also see "towers" of dark pixels at that same height, we classify decide a note is there.  (E.g., we had 10 strips in a row all see a tower of dark pixels at the same height, so we decide that there must be a note there.)

# 3. Calculate the center pixel of the note we found.

# 4. Classify double and triple notes.
a. When we find two notes that are very close together, we classify them as a double note.

b. E.g., our threshold is 10, and two centers are at (20, 56) and (25, 83).  25 - 20 = 5 < 10, so the notes are very close together and must be double notes.

c. To classify triple notes, we just need to look one note further.  If we find another note that is very close to the double note, then we classify that as a triple note.

# 5. Find pitch of first note in staff.
a. Use CNN to find pitch of that note?

b. If we use a CNN, we would cut out a window around the first note.  (The window would be the height of the image and the width of the note.  Use CNN to classify that note.

c. Note: To make this easier for the CNN, if the first note is a double note, we can instead use the first single note in the staff.

# 6. Classify pitches of all other notes based on relative height to first note.
a. We decide some threshold distance for when two notes are at different pitches, e.g., 10 pixel height difference.

b. E.g., first note is at (20, 50) and second note is at (20, 80).  80 - 50 = 30, so second note is 3 pitches above first note.

# 7. Classify note types.
a. Cut out square around each center, and use CNN to classify note type.

In [1]:
model = Sequential() 

#LAYERS:
#Convulusional layers:
#Supposedly selu is better than relu.  It's worth testing.
model.add(Conv2D(32, (3, 3), activation='selu'))
model.add(Conv2D(32, (3, 3), activation='selu'))

#Always flatten inputs after you convulusional layers are finished:
model.add(Flatten())

#Non-convolusional layers:
model.add(Dense(units=30, activation='selu', input_dim=X_train.shape[1]))
model.add(Dense(units=15, activation='selu'))

#Output Layer:
model.add(Dense(units=, activation='softmax'))

SyntaxError: invalid syntax (2375282020.py, line 17)

In [ ]:
#COMPILE THE MODEL
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
#TRAIN THE MODEL
model.fit(X_train.astype('float'), y_train, batch_size=1, epochs=10)

In [2]:
#PREDICT TEST DATA

yhat_test = model.predict(X_test.astype(float))

#We take the outcome associated with the the largest value as our result/prediction.
yhat = []
for y in yhat_test:
    yhat.append(np.argmax(y)) #Outcome associated with the largest value.
yhat = np.array(yhat)

print(yhat.shape)

NameError: name 'NoN_model' is not defined

# 8. Combine pitch and note type information to make list of all notes in staff.

# 9. Convert list to gabc output.

# 10. Evaluate Accuracy

Compare our NN's output with the gabc file from GregoBase.
Note: In gabc files, the notes are within parentheses, so we can simply pick them out.
Our output will be a list of notes represented as strings.

The accuracy function simply compares our list with the notes present in the gabc.
There are many ways to do this comparison.

The simplest is to compare the first note in our list with the first note in the gabc, then the second note in our list with the second note in the gabc, and so on.
The flaw with this is that if our scanning window misses a note, our whole accuracy measure will be thrown off.

Another way to do this is to check the quantity of each type of note in the original gabc file and see if it matches the quantity in our own output.
E.g., there are 12 g notes in the original, did our neural net find 12 g notes?

In [ ]:
def accuracy():

# Write to gabc File:

# (Optional) Staff Line Detection

In [ ]:
staff_model = Sequential()



# (Optional) Find dsl